In [2]:
import openai 
from qdrant_client import QdrantClient 

### Embedding function

In [3]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input = text,
        model = model,
    )
    return response.data[0].embedding

### Retrieval function

In [4]:
qdrant_client = QdrantClient(url="http://localhost:6333") 

In [ ]:
def retrieve_data(query, k=5): 
    query_embedding = get_embedding(query)  
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",  
        query=query_embedding, 
        limit=k,  
    ) 
    retrieved_context_ids = []
    retrieved_context = [] 
    similarity_scores = [] 
    
    for result in results.points: 
        retrieved_context_ids.append(result.payload["parent_asin"]) 
        retrieved_context.append(result.payload["description"]) 
        similarity_scores.append(result.score) 

    return {
        "retrieved_context_ids": retrieved_context_ids, 
        "retrieved_context": retrieved_context, 
        "similarity_scores": similarity_scores, 
    }


In [6]:
retrieve_data("What kind of earphones can I get?", k=10).points

[ScoredPoint(id=20, version=0, score=0.523202, payload={'description': 'Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surrounding